In [1]:
import os
import sys
import glob
import pathlib
import numpy as np
import pandas as pd
# import pylab as plt

from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE

In [2]:
sys.path.append(os.path.abspath('../src/'))
    
from model_selection import cross_val_score

from data import data_generator

In [3]:
from quality_estimator import BaseQualityEstimator

### Check metrics:

In [4]:
X, Xy, y = data_generator(root_dir="../data/decomposed/")

In [8]:
q_clf = BaseQualityEstimator(metrics=["dice_coefficient", 
#                                       "mae", "mse", 
#                                       "hausdorff_distance", 
#                                       "surface_distances",
#                                       "assd"
                                     ], 
                             unary_metrics=[],#"area"],
#                              matching_metrics=[],
                             meta_clf=LGBMClassifier(max_depth=3))

In [9]:
cross_val_score(q_clf, X, Xy, y, cv=10, scoring=MAE, random_state=4)

[0.2222222222222222,
 0.6111111111111112,
 0.7777777777777778,
 0.4444444444444444,
 0.7222222222222222,
 0.3333333333333333,
 0.6111111111111112,
 0.7222222222222222,
 0.7222222222222222,
 0.6666666666666666]

In [10]:
np.mean(_9)

0.5833333333333334

In [11]:
q_clf.fit(X_train, Xy_train, y_train)

BaseQualityEstimator(meta_clf=LGBMClassifier(max_depth=3), unary_metrics=[])

In [12]:
y_pred = q_clf.predict(X_eval, Xy_eval)

In [13]:
y_dummy = np.array([3 for i in range(len(y_eval))])

In [14]:
y_pred

array([3, 3, 4, 4, 2, 5, 4, 4, 1, 1, 4, 1, 5, 1, 2, 3, 1, 2, 1, 5, 3, 3,
       5, 4, 3, 4, 1, 4, 2, 2, 5, 4, 2, 3, 5, 5])

In [15]:
y_eval

array([3, 3, 4, 3, 3, 5, 3, 4, 1, 1, 4, 1, 5, 1, 3, 3, 2, 2, 1, 5, 4, 5,
       5, 3, 4, 5, 1, 3, 3, 4, 3, 4, 3, 5, 4, 4])

In [16]:
y_dummy

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [17]:
MAE(y_eval, y_dummy)

1.0277777777777777

In [18]:
MAE(y_eval, y_pred)

0.6111111111111112